# Step-by-step Explanation of Performing LDA on the Wine Quality Dataset

### Import the libraries

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import accuracy_score
from matplotlib import pyplot as plt
import time

### Load the dataset

In [ ]:
data = pd.read_csv('winequality.csv')
df = data.copy()
df

### Import the standardization tool

In [ ]:
scaler = StandardScaler()

In [ ]:
X = df.iloc[:,0:11].values
y = df.iloc[:,11].values

X_std = scaler.fit_transform(X)

In [ ]:
print(y)

### Calculate the class means

In [ ]:
np.set_printoptions(precision=1)

mean_vectors=[]
for gr in range(3,9):
    mean_vectors.append(np.mean(X_std[y==gr],axis=0))
    print('Mean Vector quality %s :%s\n'%(gr,mean_vectors[gr-3]))

# Calculating the Within- and Between-Class Scatter Matrices

### Calculate the within-class scatter matrix

In [ ]:
S_w = np.zeros((11,11))

for gr,mv in zip(range(3,9),mean_vectors):
    class_sc_mat=np.zeros((11,11))
    
    for row in X_std[y==gr]:
        
        row,mv = row.reshape(11,1), mv.reshape(11,1)
        class_sc_mat+=(row-mv).dot((row-mv).T)

    S_w+=class_sc_mat
print('Within class-scatter matrix:\n',S_w)

### Calculate the between-class scatter matrix

In [ ]:
overall_mean = np.mean(X_std, axis=0)
c=0
S_b=np.zeros((11,11))
for i, mean_vec in enumerate(mean_vectors):
    n=X_std[y==i+3,:].shape[0]
    c+=1
    mean_vec=mean_vec.reshape(11,1)
    overall_mean=overall_mean.reshape(11,1)
    S_b+=n*(mean_vec-overall_mean).dot((mean_vec-overall_mean).T)
  
print('Between class scatter matrix:\n',S_b)

### Find the eigenvectors and eigenvalues pairs 

In [ ]:
eig_vals,eig_vecs=np.linalg.eig(np.linalg.inv(S_w).dot(S_b))

for i in range(len(eig_vals)):
    eigvec_sc=eig_vecs[:,i].reshape(11,1)
    print('\nEigenvector {}:\n{}'.format(i+1,eigvec_sc.real))
    print('\nEigenvalue {:}:{:.2e}'.format(i+1,eig_vals[i].real))

### Order the eigenpairs in descending order with respect to the eigenvalues

In [ ]:
eig_pairs=[]
for i in range(len(eig_vals)):
    if eig_vals[i]<0:
        eig_pairs.append((-eig_vals[i],-eig_vecs[:,i]))
    else:
        eig_pairs.append((eig_vals[i],eig_vecs[:,i]))
eig_pairs=sorted(eig_pairs,key=lambda k: k[0], reverse=True)

print('Evalues in decreasing order:\n')
for i in range(len(eig_pairs)):
    print('Evalue:\n')
    print(eig_pairs[i][0].real)
    print('\n')
    print('Associated evector:\n')
    print(eig_pairs[i][1].real)
    print('\n')

# Analysis of LDA

### Show the explained variance by each respective eigenvector

In [ ]:
print('Variance explained:\n')
eigv_sum = sum(eig_vals)
for i,j in enumerate(eig_pairs):
    print('eigenvalue {0:}: {1:.2%}'.format(i+1, (j[0].real/eigv_sum.real)))  

### Take the first two eigenvectors retaining the most variance

In [ ]:
W = np.hstack((eig_pairs[0][1].reshape(11,1), eig_pairs[1][1].reshape(11,1)))
print('Matrix W:\n', W.real)

### Project the data onto the new axes (linear discriminants)

In [ ]:
X_lda = X_std.dot(W)

In [ ]:
X_std.shape

In [ ]:
X.shape

In [ ]:
X_lda.shape

### Plot the dataset after performing LDA

In [ ]:
def plot_step_lda():
    fig=plt.figure()
    
    ax = plt.subplot(111)
    
    for label,color in zip(
        range(3,9),('blue', 'red', 'green','yellow','black','orange')):

        plt.scatter(x=X_lda[:,0].real[y == label],
                y=X_lda[:,1].real[y == label],
                
               color=color,
                alpha=0.5,
                label=label
               )
       
    plt.xlabel('Linear Discriminant 1')
    plt.ylabel('Linear Discriminant 2')
    
    leg = plt.legend(loc='upper right', fancybox=True)
    leg.get_frame().set_alpha(0.5)
    plt.title('LDA: Wine data projection onto the first 2 linear discriminants')
 
    ax.spines["top"].set_visible(False)  
    ax.spines["right"].set_visible(False)
    ax.spines["bottom"].set_visible(False)
    ax.spines["left"].set_visible(False)    

    plt.grid()
    plt.show()

plot_step_lda()

### For comparison add the plot when using PCA

In [ ]:
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_std)

### Plot the dataset after perfoming PCA 

In [ ]:
def plot_step_pca():
    fig=plt.figure()

    ax = plt.subplot(111)
    for label,color in zip(
            range(3,9),('blue', 'red', 'green','yellow','black','orange')):

            plt.scatter(x=X_pca[:,0].real[y == label],
                    y=X_pca[:,1].real[y == label],
                
                    color=color,
                    alpha=0.5,
                    label=label
                    )

    plt.xlabel('Principal Component 1')
    plt.ylabel('Principal Component 2')

    leg = plt.legend(loc='upper right', fancybox=True)
    leg.get_frame().set_alpha(0.5)
    plt.title("PCA: Wine data projection onto the first 2 principal components")
    
    ax.spines["top"].set_visible(False)  
    ax.spines["right"].set_visible(False)
    ax.spines["bottom"].set_visible(False)
    ax.spines["left"].set_visible(False)    

    plt.grid()
    plt.show() 
plot_step_pca()

### Split the dataset into training and testing part

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=0)

In [ ]:
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

### Perform LDA with 2 *linear discriminants* and store the respective results from the training and testing parts

In [ ]:
lda = LDA(n_components=2)

In [ ]:
X_train_lda=lda.fit_transform(X_train,y_train)

In [ ]:
X_test_lda=lda.transform(X_test)

### Perform PCA with 2 *principal components* and store the respective results from the training and testing parts

In [ ]:
pca = PCA(n_components=2)

In [ ]:
X_train_pca=pca.fit_transform(X_train)

In [ ]:
X_test_pca=pca.transform(X_test)

### Import the classifier 

In [ ]:
svclassifier = SVC(kernel='linear')

### Run the classifier through the LDA train and test sets and return the neccessary time for training and testing

In [ ]:
def train_test_lda():
    s=[]
    start_train_lda=time.time()
    svclassifier.fit(X_train_lda,y_train)
    finish_train_lda=time.time()
    
    start_test_lda=time.time()
    y_pred_lda=svclassifier.predict(X_test_lda)
    finish_test_lda=time.time()
  
    s.append(finish_train_lda-start_train_lda)
    s.append(finish_test_lda-start_test_lda)
    s.append(y_pred_lda)

    return s

### Run the classifier through the PCA train and test sets and return the neccessary time for training and testing

In [ ]:
def train_test_pca():
    l=[]
    start_train_pca=time.time()
    svclassifier.fit(X_train_pca,y_train)
    finish_train_pca=time.time()
    
    start_test_pca=time.time()
    y_pred_pca=svclassifier.predict(X_test_pca)
    finish_test_pca=time.time()
  
    l.append(finish_train_pca-start_train_pca)
    l.append(finish_test_pca-start_test_pca)
    l.append(y_pred_pca)
    
    return l

# Analysis of the Training and Testing Times for the Classifier and Its Accuracy

In [ ]:
train_lda=0
test_lda=0
for i in range(10):
    
    c=train_test_lda()
    train_lda+=c[0]
    test_lda+=c[1]
    
print('Average time for training out of 10 runs for LDA:{}'.format(train_lda/10))
print('Average time for testing out of 10 runs for LDA:{}'.format(test_lda/10))

In [ ]:
train_pca=0
test_pca=0
for i in range(10):
    
    m=train_test_pca()
    train_pca+=m[0]
    test_pca+=m[1]
    
print('Average time for training out of 10 runs for PCA:{}'.format(train_pca/10))
print('Average time for testing out of 10 runs for PCA:{}'.format(test_pca/10))

In [ ]:
a = train_test_lda()
b = train_test_pca()

### Print out the confussion matrix for the LDA test set

In [ ]:
cm_lda = confusion_matrix(y_test,a[2])
print('Confussion matrix for LDA:\n')
print(cm_lda)
print('\n')
print('LDA Accuracy:'+' '+ str(accuracy_score(y_test,a[2])))

### Print out the confussion matrix for the PCA test set

In [ ]:
cm_pca = confusion_matrix(y_test,b[2])
print('Confussion matrix for PCA:\n')
print(cm_pca)
print('\n')
print('PCA Accuracy:'+' '+ str(accuracy_score(y_test,b[2])))